In [1]:
import sys
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/semi_supervised/')
import coreg
reload(coreg)
import rasco
reload(rasco)
import trireg
reload(trireg)
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/ordinal/')
import simple
reload(simple)
sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/ensemble/')
import stacking
reload(stacking)

sys.path.append('/afs/ee.cooper.edu/user/t/a/tam8/documents/ml_misc/neighbors/')
import rpfnn
reload(rpfnn)
import ann
reload(ann)

import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

import transformers as tforms
reload(tforms)
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union

import metrics
reload(metrics)
from sklearn.cross_validation import StratifiedKFold, train_test_split

from sklearn.base import clone

from sklearn.svm import SVC, LinearSVC, SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet, Ridge, Lasso, SGDRegressor, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier, DecisionTreeClassifier, DecisionTreeRegressor

from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.lda import LDA
from sklearn.qda import QDA

from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.isotonic import IsotonicRegression

from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_score

from collections import Counter
import minirank as mr

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from time import time

import cPickle as pickle

def wgmean(x, w):
    return np.exp(np.sum(w*np.log(x), axis=1) / np.sum(w, axis=1))

import logging 
fh = logging.FileHandler('/tmp/lmg.log')

In [ ]:
# Load Data
train_pd  = pd.read_pickle('saved/train_pd_enc.p')
test_pd  = pd.read_pickle('saved/test_pd_enc.p')
labels = pd.read_pickle('saved/labels.p')
test_ind = pickle.load(open('saved/test_ind.p'))
y_binned = np.load('saved/y_binned.npy')

fi = np.load('saved/feature_importances.npy')
y_binned[y_binned==6] = 5

drop_cols = ['T1_V10', 'T1_V13', 'T2_V7', 'T2_V10']
# drop_cols = train_pd.columns[fi < 0.01]


for col in drop_cols:
    train_pd.drop(col, axis=1, inplace=True)
    test_pd.drop(col, axis=1, inplace=True)


In [2]:
# Load Data custom
train_pd  = pd.read_pickle('saved/train_pd_custom.p')
test_pd  = pd.read_pickle('saved/test_pd_custom.p')

labels = pd.read_pickle('saved/labels.p')
test_ind = pickle.load(open('saved/test_ind.p'))
y_binned = np.load('saved/y_binned.npy')

In [12]:
# Load Data le instead
# train_pd  = pd.read_pickle('saved/train_pd_l_enc.p')
# test_pd  = pd.read_pickle('saved/test_pd_l_enc.p')

train_pd  = pd.read_pickle('saved/train_pd_le_and_oh_enc.p')
test_pd  = pd.read_pickle('saved/test_pd_le_and_oh_enc.p')

labels = pd.read_pickle('saved/labels.p')
test_ind = pickle.load(open('saved/test_ind.p'))
y_binned = np.load('saved/y_binned.npy')
fi = np.load('saved/feature_importances.npy')

y_binned[y_binned==6] = 5

drop_cols = ['T1_V10', 'T1_V13', 'T2_V7', 'T2_V10']
# drop_cols = []

# drop_cols = train_pd.columns[fi < 0.01]


for col in drop_cols:
    train_pd.drop(col, axis=1, inplace=True)
    test_pd.drop(col, axis=1, inplace=True)

In [ ]:
# Load Data binary instead
train_pd  = pd.read_pickle('saved/train_pd_binary_enc.p')
test_pd  = pd.read_pickle('saved/test_pd_binary_enc.p')
labels = pd.read_pickle('saved/labels.p')
test_ind = pickle.load(open('saved/test_ind.p'))
y_binned = np.load('saved/y_binned.npy')

y_binned[y_binned==6] = 5

In [13]:
X_tsne2_26 = np.load('saved/X_tsne2_26important.npy')
X_tsne2_26_train = X_tsne2_26[:len(train_pd), :]
X_tsne2_26_test = X_tsne2_26[-len(test_pd):, :]

X_tsne2 = np.load('saved/X_tsne2.npy')
X_tsne2_train = X_tsne2[:len(train_pd), :]
X_tsne2_test = X_tsne2[-len(test_pd):, :]

print X_tsne2_train.shape
print X_tsne2_test.shape
print X_tsne2_26_train.shape
print X_tsne2_26_test.shape

(50999, 2)
(51000, 2)
(50999, 2)
(51000, 2)


In [4]:
# X_fa10_train = np.load('saved/X_fa10_train.npy')
# X_fa10_test = np.load('saved/X_fa10_test.npy')
# print X_fa10_train.shape, X_fa10_test.shape

X_fa40_train = np.load('saved/X_fa40_train.npy')
X_fa40_test = np.load('saved/X_fa40_test.npy')
print X_fa40_train.shape, X_fa40_test.shape

(50999, 38) (51000, 38)


In [3]:
train = np.array(train_pd)
test = np.array(test_pd)

X_train = train.astype(float)
X_test = test.astype(float)
y_train = np.array(labels)

try:
    X_train = np.c_[X_train, X_tsne2_26_train]
    X_test = np.c_[X_test, X_tsne2_26_test]
#     X_train = X_tsne2_26_train
#     X_test = X_tsne2_26_test
except:
    pass

try:
    X_train = np.c_[X_train, X_3000mean]
except:
    pass

holdout = False
if holdout:
    X_train, X_hold, \
    y_train, y_hold, \
    y_binned, y_binned_hold \
    = train_test_split(
        X_train, y_train, y_binned, 
        test_size=0.2, random_state=0)

# """
pipe_x = make_pipeline(
    make_union(
        tforms.IdentityTformer(),
#         make_pipeline(AddTformer(1), BoxCoxTformer()),
#         AnscombeTformer(),
    ),
#     StandardScaler(),
)
pipe_y = make_pipeline(
    tforms.IdentityTformer(),
#     tforms.BoxCoxTformer(),
#     tforms.LogTformer(),
#     tforms.AnscombeTformer(),
#     tforms.FreemanTukeyTformer(),
#     tforms.ArcsinhTformer(),
#     StandardScaler(),
    
)
pipe_x.fit(np.r_[X_train, X_test])
pipe_y.fit(y_train)

X_train = pipe_x.transform(X_train)
X_test = pipe_x.transform(X_test)
try:
    X_hold = pipe_x.transform(X_hold)
except:
    pass
# y_train = pipe_y.fit_transform(y_train)

# """

# small_n = 5000
# X_train = X_train[:small_n,:]
# y_train = y_train[:small_n]
# y_binned = y_binned[:small_n]

print 'y_train', y_train.shape
print 'X_train', X_train.shape
print 'X_test', X_test.shape
try:
    print 'X_hold', X_hold.shape
except:
    pass
print len(np.unique(y_train))
print len(np.unique(y_binned))
print type(X_train[0][0])

y_train (50999,)
X_train (50999, 79)
X_test (51000, 79)
X_hold 50
5
<type 'numpy.float64'>


In [5]:
# from sklearn.datasets import dump_svmlight_file, load_svmlight_file
from svmlight_loader import dump_svmlight_file, load_svmlight_file
mapped_train, _y = load_svmlight_file('saved/mapped2000_train.libsvm')

X_2000mean = mapped_train.todense()

In [6]:
X_nn = np.c_[X_2000mean, X_train]
X_nn.shape

(50999, 2125)

In [8]:
# Keras

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, MaxoutDense, Reshape
from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.optimizers import Adadelta, Adagrad, Adam, RMSprop
from keras.layers.advanced_activations import ParametricSoftplus, PReLU

n_feats = X_nn.shape[1]
drop_prob = 0.6

model = Sequential()


hidden_size = 1024
model.add(Dense(n_feats, hidden_size))
model.add(Dropout(0.2))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, hidden_size))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, hidden_size))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, hidden_size))
model.add(ParametricSoftplus(hidden_size))
model.add(BatchNormalization((hidden_size,)))
model.add(Dropout(drop_prob))
model.add(Dense(hidden_size, 1))
model.add(Activation('linear'))

loss_type = 'msle'

opt = RMSprop(lr=0.0005, rho=0.75, epsilon=1e-6)

# model.compile(loss=loss_type, optimizer=opt)
# model.save_weights('saved/nn_weights', overwrite=True)

clf_nn = KerasRegressor(model, optimizer=opt, loss=loss_type,
                       train_batch_size=1024*4, test_batch_size=1024*16,
                       nb_epoch=500)


In [116]:
# GBM models

gbm = xgb.XGBRegressor(
    objective="reg:linear",
    n_estimators=900,
    learning_rate=0.005,
#     gamma=0.0,
    max_depth=9,
    min_child_weight=6,
#     max_delta_step=10,
    subsample=0.7,
    colsample_bytree=0.7,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=7,
    seed=np.random.randint(0,100),
    silent=True,
)

meta_gbm = xgb.XGBRegressor(
    objective="reg:linear",
    n_estimators=750,
    learning_rate=0.005,
#     gamma=0.0,
    max_depth=9,
    min_child_weight=6,
#     max_delta_step=10,
    subsample=0.7,
    colsample_bytree=0.7,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=7,
    seed=np.random.randint(0,100),
    silent=True,
)



gbm_bin = xgb.XGBClassifier(
    objective="binary:logistic",
    n_estimators=500,
    learning_rate=0.01,
#     gamma=0.0,
    max_depth=8,
    min_child_weight=5,
#     max_delta_step=10,
    subsample=0.8,
    colsample_bytree=0.8,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=8,
    seed=np.random.randint(0,100),
    silent=True,
)

gbm_cat = xgb.XGBClassifier(
    objective="binary:softmax",
    n_estimators=500,
    learning_rate=0.01,
#     gamma=0.0,
    max_depth=8,
    min_child_weight=5,
#     max_delta_step=10,
    subsample=0.8,
    colsample_bytree=0.8,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=8,
    seed=np.random.randint(0,100),
    silent=True,
)

gbm_fast = xgb.XGBClassifier(
    objective="binary:softmax",
    n_estimators=50,
    learning_rate=0.3,
#     gamma=0.0,
    max_depth=8,
    min_child_weight=5,
#     max_delta_step=10,
    subsample=1.0,
    colsample_bytree=1.0,
#     scale_pos_weight=1.0,
    base_score=0.5,
    nthread=8,
    seed=np.random.randint(0,100),
    silent=True,
)

In [182]:
from sklearn.base import clone
%pdb off

rasco_nb = rasco.Rasco(base_estimator=GaussianNB(),
                       n_estimators=32, max_iters=20,
                       verbose=True,
                      n_jobs=-1)

rasco_dt = rasco.Rasco(base_estimator=DecisionTreeClassifier(),
                       n_estimators=128, max_iters=20,
                       verbose=True,
                      n_jobs=-1)

rasco_gbm = rasco.Rasco(base_estimator=gbm_cat,
                       n_estimators=8, max_iters=20,
                       verbose=True,
                      n_jobs=1)
rasco_logistic = rasco.Rasco(base_estimator=LogisticRegression(),
                            n_estimators=8, max_iters=10,
                            verbose=True,
                            n_jobs=-1)

h = [GaussianNB() for _ in range(64)] + \
    [DecisionTreeClassifier(max_depth=6, min_samples_split=5, class_weight='auto') for _ in range(16)] + \
    [SGDClassifier(n_iter=20, loss='modified_huber', class_weight='auto', n_jobs=1) 
     for _ in range(8)] + \
    [ExtraTreeClassifier(max_depth=6, min_samples_split=6, class_weight='auto')
     for _ in range(16)]


rasco_mix = rasco.Rasco(h,
                        feat_ratio=0.6,
                       n_estimators=104, max_iters=5,
                       verbose=True,
                      n_jobs=-1)

ord_rasco = simple.SimpleOrdinalClassifier(rasco_mix, n_jobs=1)

ord_gbm_bin = simple.SimpleOrdinalClassifier(gbm_bin, n_jobs=1)
ord_gaussnb_bin = simple.SimpleOrdinalClassifier(GaussianNB(), n_jobs=-1)
ord_logistic_bin = simple.SimpleOrdinalClassifier(LogisticRegression(), n_jobs=-1)
ord_rf_bin = simple.SimpleOrdinalClassifier(RandomForestClassifier(n_estimators=200, n_jobs=-1), n_jobs=1)
ord_ada_bin = simple.SimpleOrdinalClassifier(AdaBoostClassifier(n_estimators=200), n_jobs=-1)
ord_dt = simple.SimpleOrdinalClassifier(DecisionTreeClassifier(), n_jobs=-1)
ord_lda = simple.SimpleOrdinalClassifier(LDA(solver='lsqr'), n_jobs=-1)

# clf0 = simple.SimpleOrdinalClassifier(base_estimator_type=GaussianNB, base_estimator_params={}, n_jobs=1)
# clf0 = simple.SimpleOrdinalClassifier(SVC(probability=True), n_jobs=8)
# clf0 = simple.SimpleOrdinalClassifier(clf_nn, n_jobs=1)
# clf0 = simple.SimpleOrdinalClassifier(base_estimator_type=KerasClassifier,
#                                       base_estimator_params=nn_params,
#                                       label_transformer=label_bin_tform,
#                                       n_jobs=1)

rf = RandomForestRegressor(n_jobs=-1, **{'max_leaf_nodes': None, 
                                         'bootstrap': False, 
                                         'min_samples_leaf': 8, 
                                         'n_estimators': 150, 
                                         'min_samples_split': 8, 
                                         'min_weight_fraction_leaf': 0.25, 
                                         'max_features': 'sqrt', 
                                         'max_depth': 12} )

et = ExtraTreesRegressor(n_jobs=-1, **{'max_leaf_nodes': None, 
                                         'bootstrap': False, 
                                         'min_samples_leaf': 8, 
                                         'n_estimators': 150, 
                                         'min_samples_split': 8, 
                                         'min_weight_fraction_leaf': 0.25, 
                                         'max_features': 'sqrt', 
                                         'max_depth': 12} )

stack = stacking.Stacking([
        ord_gaussnb_bin,
        ord_lda,
        rf,
        clf_nn,
#         ord_rf_bin,
#         ord_ada_bin,
#         ord_logistic_bin,
#         ord_gbm_bin,
#         ann.ann(no_trees=50, num_neighbors=100, weights='distance'),
#         KNeighborsRegressor(n_neighbors=20),

        LinearRegression(),
        Ridge(), 
#         BaggingRegressor(n_estimators=400, max_features=0.8, n_jobs=-1),
#         ExtraTreesRegressor(n_estimators=400, n_jobs=-1), 
        
#         gbm,
    ],
                          meta_gbm,
#                           LinearRegression(),
#                           gbm,
                          fit_params={
        'base0_y': 'y_binned',
        'base1_y': 'y_binned',
        'base3_X': 'X_nn_train',
#         'base3_fit_batch_size': 1024*16,
#         'base3_predict_batch_size': 1024*16,
#         'base3_fit_nb_epoch': 500,
        
#         'base2_y': 'y_binned',
    },
                          pred_params={
        'base3_X': 'X_nn_val',
#         'base3_predict_batch_size': 1024*16,
        
#         'base0_weighted':True,
#         'base1_weighted':True,
    },
                          include_orig_feats=False,
                          use_probs=True,
                          cv=8,
                          verbose=1,
                          save_level0_out=True,
                          log_handler=fh,
                         )
# clfs = [clf0, gbm] 
# clfs = [clone(gbm), stack]
# clfs = [LinearRegression(), stack]
# clfs = [stack]
# clfs = [BaggingRegressor(n_estimators=200, max_features=0.8, n_jobs=-1)]

# clfs = [stack]
clfs = [ord_rasco]
# clfs = [rasco_mix]
# clfs = []
# clfs = [LinearRegression()]
# clfs = [rf]



# clfs = [rpfnn.rpfnn(leaf_size=100, no_trees=10, num_neighbors=100)]
# clfs = [ann.ann(no_trees=50, num_neighbors=100, weights='distance')]

# clfs = [ord_gaussnb_bin]
# clfs = [ord_lda]
# clfs = [simple.SimpleOrdinalClassifier(LDA(solver='lsqr'), n_jobs=-1)]
# clfs = [clf1, clf2, ]


# h = [clone(clf1), clone(clf2)]
# clf = coreg.CoReg(h=h, T=5, verbose=True, n_jobs=1)

scores = []
scores_base = []
lols = []
n_reps = 1
k = 3
for reps in range(n_reps):
    skf = StratifiedKFold(y_binned, n_folds=k,
                          shuffle=True,
                          random_state=np.random.randint(0,100))
    for ii, (train, valid) in enumerate(skf):
        
        
#         history = LossHistory()   # for keras
        print 'Fold %d' % ii,
        X_train_k, X_valid_k = X_train[train], X_train[valid]
#         X_train_k, X_valid_k = X_fa40_train[train], X_fa40_train[valid]
        y_train_k, y_valid_k = y_train[train], y_train[valid]
        y_train_binned_k, y_valid_binned_k = y_binned[train], y_binned[valid]
        
        X_nn_train_k, X_nn_valid_k = X_nn[train], X_nn[valid]
        
        
        X_all = np.r_[X_train_k, X_valid_k]
        y_all = np.r_[y_train_k, np.nan*np.ones(len(y_valid_k))]
        y_binned_all = np.r_[y_train_binned_k, np.nan*np.ones(len(y_valid_binned_k))]
        
        tic = time()
#         clf.fit(X_train_k, y_train_k)
#         clf.fit(X_all, y_all)


        stack.extra_data['y_binned'] = y_train_binned_k
        stack.extra_data['X_nn_train'] = X_nn_train_k
        stack.extra_data['X_nn_val'] = X_nn_valid_k
        
        rasco_mix.y_val = y_valid_binned_k - 1
        for clf in clfs:
            clf.fit(X_all, y_binned_all-1)
#             clf.fit(X_train_k, pipe_y.transform(y_train_k))
#             clf.fit(X_train_k, y_train_binned_k-1)
#             clf.fit(X_train_k, y_train_binned_k,
#                    nb_epoch=60, batch_size=1024*16, )
#             clf.fit(X_train_k, y_train_k, 
#                     base0_y=y_train_binned_k,
#                     base1_y=y_train_binned_k,
# #                     base2_y=y_train_binned_k,
#                    )
    
    
        
        # Minirank
#         w, theta = mr.ordinal_logistic_fit(X_train_k, y_train_k, verbose=False,
#                                 solver='TNC')

        
        toc = time() - tic
        print 'Train time: %2.3f s\t' % toc,
        tic = time()

        
        
#         valid_preds = [clf.predict_weighted(X_valid_k,) 
#                        if hasattr(clf, 'predict_weighted') 
#                        else clf.predict(X_valid_k)
#                        for clf in clfs]
        
        valid_preds = [clf.predict(X_valid_k) for clf in clfs]
        
#         valid_preds = [clf.predict_weighted(X_valid_k, batch_size=1024*16) 
#                if hasattr(clf, 'predict_weighted') 
#                else clf.predict(X_valid_k)
#                for clf in clfs]
        
        try:
            lol = metrics.normalized_gini(y_valid_k, 
                                          clfs[0].predict_weighted(X_valid_k, geometric=True))
            lols.append(lol)
        except:
            pass

        # Minirank
#         valid_preds = mr.ordinal_logistic_predict(w, theta, X_valid_k)
        
        valid_base_preds = np.mean([clf.predict(X_valid_k) for clf in clfs
                                   ], 
                                   axis=0)

        
        score = [metrics.normalized_gini(y_valid_k, v) for v in valid_preds]
        score_base = metrics.normalized_gini(y_valid_k, valid_base_preds)
        
        toc_pred = time() - tic
        print 'Pred time: %2.3f s\t' % toc_pred, 
        
        print 'Score', score
        print 'Score avg ensemble', score_base
        scores.append(score)
        scores_base.append(score_base)
        
#         break
        
        
print "done"
print 'Score:', np.array(scores).mean(axis=0)
print 'Base Score:', np.array(scores_base).mean()
print 'lol', lols, np.mean(lols)

INFO:rasco:RASCO Init
INFO:rasco:RASCO Init
INFO:rasco:RASCO Init
INFO:rasco:RASCO Init
INFO:rasco:RASCO Init
INFO:stacker:Stacker Init
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.901985
DEBUG:rasco:Iter #0 time: 5.80886
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.908127
DEBUG:rasco:Iter #1 time: 5.84686
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.894073
DEBUG:rasco:Iter #2 time: 6.03353
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.893093
DEBUG:rasco:Iter #3 time: 5.94141
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.901929
DEBUG:rasco:Iter #4 time: 5.91349
DEBUG:rasco:Total time: 29.5478
DEBUG:rasco:Best candidate: pred_class=1 | Prob: 0.903516
DEBUG:rasco:Iter #0 time: 5.80821
DEBUG:rasco:Best candidate: pred_class=1 | Prob: 0.914373
DEBUG:rasco:Iter #1 time: 5.91069
DEBUG:rasco:Best candidate: pred_class=1 | Prob: 0.909786
DEBUG:rasco:Iter #2 time: 5.9186
DEBUG:rasco:Best candidate: pred_class=1 | Prob: 0.920266
DEBUG:rasco:Iter #3 time: 5.92052
DEBUG

Automatic pdb calling has been turned OFF
Fold 0 Train time: 117.654 s	Pred time: 33.352 s	

DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.914504
DEBUG:rasco:Iter #0 time: 5.81148
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.903988
DEBUG:rasco:Iter #1 time: 5.93031
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.902313
DEBUG:rasco:Iter #2 time: 5.96014
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.910863
DEBUG:rasco:Iter #3 time: 5.94718
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.90813
DEBUG:rasco:Iter #4 time: 5.92928
DEBUG:rasco:Total time: 29.582
DEBUG:rasco:Best candidate: pred_class=1 | Prob: 0.908552
DEBUG:rasco:Iter #0 time: 5.81869
DEBUG:rasco:Best candidate: pred_class=1 | Prob: 0.904668
DEBUG:rasco:Iter #1 time: 5.92255
DEBUG:rasco:Best candidate: pred_class=1 | Prob: 0.910663
DEBUG:rasco:Iter #2 time: 5.93496
DEBUG:rasco:Best candidate: pred_class=1 | Prob: 0.905872
DEBUG:rasco:Iter #3 time: 5.97166
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.898121
DEBUG:rasco:Iter #4 time: 5.91031
DEBUG:rasco:Total time: 29.5619
DEBUG:rasco:Best c

Score [0.20677959061209183]
Score avg ensemble 0.206779590612
Fold 1 Train time: 117.999 s	Pred time: 33.515 s	

DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.920834
DEBUG:rasco:Iter #0 time: 5.94445
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.904558
DEBUG:rasco:Iter #1 time: 5.93389
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.908231
DEBUG:rasco:Iter #2 time: 5.90467
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.904302
DEBUG:rasco:Iter #3 time: 5.9072
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.926705
DEBUG:rasco:Iter #4 time: 6.03077
DEBUG:rasco:Total time: 29.7246
DEBUG:rasco:Best candidate: pred_class=1 | Prob: 0.915491
DEBUG:rasco:Iter #0 time: 5.81465
DEBUG:rasco:Best candidate: pred_class=1 | Prob: 0.910574
DEBUG:rasco:Iter #1 time: 5.93479
DEBUG:rasco:Best candidate: pred_class=1 | Prob: 0.902904
DEBUG:rasco:Iter #2 time: 5.95958
DEBUG:rasco:Best candidate: pred_class=0 | Prob: 0.886849
DEBUG:rasco:Iter #3 time: 5.91066
DEBUG:rasco:Best candidate: pred_class=1 | Prob: 0.907669
DEBUG:rasco:Iter #4 time: 5.95429
DEBUG:rasco:Total time: 29.5776
DEBUG:rasco:Best 

Score [0.19520612253275124]
Score avg ensemble 0.195206122533
Fold 2 Train time: 118.100 s	Pred time: 33.402 s	Score [0.20393425330556705]
Score avg ensemble 0.203934253306
done
Score: [ 0.20197332]
Base Score: 0.20197332215
lol [-0.008757804121674033, -0.0022976210660010515, -0.01221072178961156] -0.00775538232576


In [172]:
clf = clfs[0]
q = clf.predict_proba(X_valid_k)

array([1, 1, 7, ..., 4, 3, 9])

In [ ]:
est = stack.base_estimators[3]
est.config_['layers'][-1]['output_dim']

In [ ]:
print y_train_k.shape
print y_train_binned_k.shape
# gbm is ~ 0.381  @ 30 sec per fold?
# gbm on binned labels ~ 0.378 @ 30 sec per fold... ~_~
# ordinal nn binned is like 0.34 with stdscaling X 60 epochs

# the stack is worse?????? 0.379
# added nn
# stack including features meta gbm 0.369
# move gbm to base and dont includebase ests (use linreg for meta) -> 0.377
# increase folds cv=8 -> 0.379

In [9]:

print 'Score:', np.array(scores).mean(axis=0)
print 'Base Score:', np.array(scores_base).mean()
print 'lol', lols, np.mean(lols)

Score: [ 0.37916189]
Base Score: 0.379161888136
lol [] nan


In [ ]:
metrics.normalized_gini(y_valid_k, clfs[0].predict_weighted(X_valid_k, geometric=True))

In [ ]:
print y_pred_base_hold.shape
print y_hold[:, None].shape

In [11]:
X_stack = stack.level0_out[0]
y_stack = stack.level0_out[1]
print X_stack.shape
print y_stack.shape

(33998, 14)
(33998,)


In [23]:
print y_train.mean()
print y_stack.mean()
print y_train_k.mean()

4.02278476049
2.35284428496
2.35945290095


In [40]:
params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.005
params["min_child_weight"] = 6
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
# params["max_delta_step"] = 10
params["silent"] = 1
params["max_depth"] = 9 #7
# params["gamma"] = 0


plst = list(params.items())

max_rounds = 2000
# xgtest = xgb.DMatrix(X_test)


skf = StratifiedKFold(y_stack, n_folds=3,
                      shuffle=True,
                      random_state=np.random.randint(0,100))
train, valid= iter(skf).next()
X_train_k = X_stack[train]
X_valid_k = X_stack[valid]

y_train_k = y_stack[train]
y_valid_k = y_stack[valid]


#create a train and validation dmatrices 
xgtrain = xgb.DMatrix(X_train_k, label=y_train_k)
xgval = xgb.DMatrix(X_valid_k, label=y_valid_k)

#train using early stopping and predict
watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(plst, xgtrain, max_rounds, watchlist, early_stopping_rounds=120)

preds_val = model.predict(xgval)

s = metrics.normalized_gini(y_valid_k, preds_val)

print s


print metrics.normalized_gini(
    y_valid_k, 
    LinearRegression().fit(X_train_k, y_train_k).predict(X_valid_k))

Will train until val error hasn't decreased in 120 rounds.
[0]	train-rmse:2.233049	val-rmse:2.233330
[1]	train-rmse:2.224972	val-rmse:2.225478
[2]	train-rmse:2.216940	val-rmse:2.217663
[3]	train-rmse:2.208930	val-rmse:2.209854
[4]	train-rmse:2.201055	val-rmse:2.202188
[5]	train-rmse:2.193213	val-rmse:2.194567
[6]	train-rmse:2.185442	val-rmse:2.187056
[7]	train-rmse:2.177714	val-rmse:2.179523
[8]	train-rmse:2.169972	val-rmse:2.172019
[9]	train-rmse:2.162337	val-rmse:2.164564
[10]	train-rmse:2.154759	val-rmse:2.157186
[11]	train-rmse:2.147175	val-rmse:2.149861
[12]	train-rmse:2.139671	val-rmse:2.142588
[13]	train-rmse:2.132219	val-rmse:2.135376
[14]	train-rmse:2.124771	val-rmse:2.128183
[15]	train-rmse:2.117374	val-rmse:2.121008
[16]	train-rmse:2.110065	val-rmse:2.113907
[17]	train-rmse:2.102777	val-rmse:2.106839
[18]	train-rmse:2.095571	val-rmse:2.099890
[19]	train-rmse:2.088372	val-rmse:2.092894
[20]	train-rmse:2.081231	val-rmse:2.085951
[21]	train-rmse:2.074061	val-rmse:2.079023
[22]	

0.301873235152
0.318391809956


Stopping. Best iteration:
[770]	train-rmse:1.051418	val-rmse:1.205652



In [28]:
# normal
from transformers import *

params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.005
params["min_child_weight"] = 6
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
# params["max_delta_step"] = 1
params["silent"] = 1
params["max_depth"] = 5 #7
# params["scale_pos_weight"] = 1.0
# params["gamma"] = 0


plst = list(params.items())

max_rounds = 5000
# xgtest = xgb.DMatrix(X_test)


skf = StratifiedKFold(y_binned, n_folds=8,
                      shuffle=True,
                      random_state=np.random.randint(0,100))
qq_scores = []
for train, valid in skf:
# train, valid= iter(skf).next()
    X_train_k = X_train[train]
    X_valid_k = X_train[valid]
#     y_train_k = GeneralTformer(lambda x: np.log(x)).transform(y_train[train])
#     y_valid_k = GeneralTformer(lambda x: np.log(x)).transform(y_train[valid])
    
    y_train_k = y_binned[train]
    y_valid_k = y_binned[valid]
    
    y_valid_k_orig = y_train[valid]


    #create a train and validation dmatrices 
    xgtrain = xgb.DMatrix(X_train_k, label=y_train_k)
    xgval = xgb.DMatrix(X_valid_k, label=y_valid_k)

    #train using early stopping and predict
    watchlist = [(xgtrain, 'train'),(xgval, 'val')]
    model = xgb.train(plst, xgtrain, max_rounds, watchlist, 
                      early_stopping_rounds=100,
                      verbose_eval=False)

    preds_val = model.predict(xgval)

    s = metrics.normalized_gini(y_valid_k_orig, preds_val)
    print s
    qq_scores.append(s)

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[2082]	train-rmse:1.154127	val-rmse:1.200988

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[2759]	train-rmse:1.142831	val-rmse:1.192744



0.3831564061
0.38269873267

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[2550]	train-rmse:1.146314	val-rmse:1.196913




0.383180011154

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[3556]	train-rmse:1.130771	val-rmse:1.191296



...


0.387681762343

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[2141]	train-rmse:1.153572	val-rmse:1.195032




0.376591462306

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[2983]	train-rmse:1.141272	val-rmse:1.184672




0.4018301357


In [13]:
# rank
from transformers import *

params = {}
params["objective"] = "rank:pairwise"
params["eta"] = 0.0005
# params["min_child_weight"] = 6
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
# params["max_delta_step"] = 1
params["silent"] = 1
params["max_depth"] = 5 #7
# params["scale_pos_weight"] = 1.0
# params["gamma"] = 0


plst = list(params.items())

max_rounds = 5000
# xgtest = xgb.DMatrix(X_test)


skf = StratifiedKFold(y_binned, n_folds=8,
                      shuffle=True,
                      random_state=np.random.randint(0,100))
qq_scores = []
for train, valid in skf:
# train, valid= iter(skf).next()
    X_train_k = X_train[train]
    X_valid_k = X_train[valid]
#     y_train_k = GeneralTformer(lambda x: np.log(x)).transform(y_train[train])
#     y_valid_k = GeneralTformer(lambda x: np.log(x)).transform(y_train[valid])
    
    y_train_k = y_binned[train]
    y_valid_k = y_binned[valid]
    
    y_valid_k_orig = y_train[valid]


    #create a train and validation dmatrices 
    xgtrain = xgb.DMatrix(X_train_k, label=y_train_k)
    xgval = xgb.DMatrix(X_valid_k, label=y_valid_k)

    #train using early stopping and predict
    watchlist = [(xgtrain, 'train'),(xgval, 'val')]
    model = xgb.train(plst, xgtrain, max_rounds, watchlist, 
                      early_stopping_rounds=100,
                      verbose_eval=False)

    preds_val = model.predict(xgval)

    s = metrics.normalized_gini(y_valid_k_orig, preds_val)
    print s
    qq_scores.append(s)
#     break

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[0]	train-map:1.000000	val-map:1.000000

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[0]	train-map:1.000000	val-map:1.000000



0.311622989463
0.356664424286

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[0]	train-map:1.000000	val-map:1.000000




0.327004161352

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[0]	train-map:1.000000	val-map:1.000000



...


0.326850757251

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[0]	train-map:1.000000	val-map:1.000000




0.324704630157

Will train until val error hasn't decreased in 100 rounds.
Stopping. Best iteration:
[0]	train-map:1.000000	val-map:1.000000




0.322748426445


In [14]:
print qq_scores
print np.mean(qq_scores)

# X_train

# inverse log shit
# [0.39597194582097206, 0.4020202827906678, 0.3824776199022992, 0.3900430760258498, 0.35743296939952146, 0.39191686671188475, 0.3482977003794325, 0.381895996324256]
# 0.381257057169
# [912]	train-rmse:3.202105	val-rmse:3.772708

# X_fa10
# [708]	train-rmse:3.526680	val-rmse:3.874051

# np.c_[X_fa10_train, X_train]
# [986]	train-rmse:3.106901	val-rmse:3.801840

# X_fa40_train
# [1071]	train-rmse:3.103334	val-rmse:3.839233

#np.c_[X_fa40_train, X_train]
#[1222]	train-rmse:2.904194	val-rmse:3.807806


# [0.38880619454668014, 0.38172734639160605, 0.37474434984100036]
# 0.381759296926

# normal 8 fold
# [0.3731329094124217, 0.3866396348815107, 0.408353381366699, 0.39934839719669324, 0.3876562009578491, 0.38201529516777405, 0.381116988214003, 0.3726814279504832]
# 0.386368029393

# inverse log shit
# [0.39597194582097206, 0.4020202827906678, 0.3824776199022992, 0.3900430760258498, 0.35743296939952146, 0.39191686671188475, 0.3482977003794325, 0.381895996324256]
# 0.381257057169

# log
# [0.4061604139576095, 0.39974187168297554, 0.3695108303351937, 0.3766801117504891, 0.39626831898568365, 0.39644198930580565, 0.37456796456045366, 0.3822501999865891]
# 0.387702712571

# stupid np.log(2-1./np.log(x+1))
# [0.3738971433322404, 0.3820780018728879, 0.37198882562261215, 0.36405897469707776, 0.3517941599099296, 0.38915260748210284, 0.3889183047847193, 0.37215531550206715]
# 0.37425541665

# binned
# [0.3831564061001914, 0.38269873267014587, 0.3831800111542416, 0.38907625052759365, 0.39850063450478496, 0.387681762343047, 0.3765914623062067, 0.4018301357001798]
# 0.387839424413

[0.3116229894629838, 0.35666442428583095, 0.32700416135203253, 0.33032248911117384, 0.3014149818678528, 0.32685075725051166, 0.3247046301574321, 0.32274842644538476]
0.325166607492


In [43]:
%%time

q = cross_val_score(DecisionTreeRegressor(), 
                    X_train, y_train, 
                    scoring=metrics.gini_score,
                    cv=skf, n_jobs=-1)
print np.mean(q)

0.126527400315
CPU times: user 807 ms, sys: 693 ms, total: 1.5 s
Wall time: 6.2 s


In [170]:
# gbm categorical

params = {}
params["objective"] = "multi:softmax"
params["num_class"] = len(np.unique(y_binned))
params["eta"] = 0.3
params["min_child_weight"] = 6
params["subsample"] = 1.0
params["colsample_bytree"] = 1.0
# params["max_delta_step"] = 10
params["silent"] = 1
params["max_depth"] = 6 #7
# params["gamma"] = 0


plst = list(params.items())

max_rounds = 2000
# xgtest = xgb.DMatrix(X_test)


skf = StratifiedKFold(y_binned, n_folds=3,
                      shuffle=True,
                      random_state=np.random.randint(0,100))
train, valid= iter(skf).next()
X_train_k = X_train[train]
X_valid_k = X_train[valid]
y_train_k = (y_binned-1)[train]
y_valid_k = (y_binned-1)[valid]


# create a train and validation dmatrices 
xgtrain = xgb.DMatrix(X_train_k, label=y_train_k)
xgval = xgb.DMatrix(X_valid_k, label=y_valid_k)

# train using early stopping and predict
watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(plst, xgtrain, max_rounds, watchlist, early_stopping_rounds=30)

preds_val = model.predict(xgval)

s = metrics.normalized_gini(y_valid_k, preds_val)
print s

Will train until val error hasn't decreased in 30 rounds.
[0]	train-merror:0.603477	val-merror:0.628669
[1]	train-merror:0.595741	val-merror:0.627140
[2]	train-merror:0.593329	val-merror:0.625846
[3]	train-merror:0.590652	val-merror:0.626081
[4]	train-merror:0.586240	val-merror:0.625257
[5]	train-merror:0.581328	val-merror:0.624551
[6]	train-merror:0.580005	val-merror:0.623375
[7]	train-merror:0.579181	val-merror:0.622493
[8]	train-merror:0.575828	val-merror:0.623022
[9]	train-merror:0.571975	val-merror:0.623610
[10]	train-merror:0.569739	val-merror:0.622963
[11]	train-merror:0.566739	val-merror:0.623963
[12]	train-merror:0.564504	val-merror:0.622787
[13]	train-merror:0.560915	val-merror:0.623610
[14]	train-merror:0.557445	val-merror:0.623375
[15]	train-merror:0.554680	val-merror:0.623846
[16]	train-merror:0.552591	val-merror:0.624610
[17]	train-merror:0.551003	val-merror:0.623728
[18]	train-merror:0.548621	val-merror:0.625316
[19]	train-merror:0.545826	val-merror:0.624551
[20]	train-m

0.191175929337


Stopping. Best iteration:
[40]	train-merror:0.501765	val-merror:0.621375



In [98]:
np.unique(y_binned)

array([1, 2, 3, 4, 5])

# Forest Neighbors

In [ ]:
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint

reload(rpfnn)
clf = rpfnn.rpfnn(leaf_size=50, no_trees=10, num_neighbors=10)

param_dist = {"leaf_size": sp_randint(20, 500),
              "no_trees": sp_randint(10, 400),
              "num_neighbors": sp_randint(1, 200),
             }

n_iter_search = 20
random_search = RandomizedSearchCV(clf, 
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   cv=skf,
                                   scoring=metrics.gini_score,
                                   n_jobs=-1)

In [ ]:
random_search.fit(X_train, y_train)


In [ ]:
print random_search.best_score_
print random_search.best_params_

# Param Finding

In [ ]:
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

k = 3
skf = StratifiedKFold(y_binned, n_folds=k,
                          shuffle=True,
                          random_state=np.random.randint(0,100))

# clf = RandomForestRegressor()
clf = ExtraTreesRegressor()


param_dist = {"n_estimators": sp_randint(10, 200),
              "max_features": ['auto', 'sqrt', 'log2'],
              "max_depth": [None] + range(1, 15),
              "min_samples_split": sp_randint(2, 10),
              "min_samples_leaf": sp_randint(1, 10),
              "min_weight_fraction_leaf": sp_uniform(0,0.5),
              "max_leaf_nodes": [None] + range(2, 20),
              "bootstrap": [True, False],
             }



n_iter_search = 256
random_search = RandomizedSearchCV(clf, 
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   cv=skf,
                                   scoring=metrics.gini_score,
                                   n_jobs=-1,
                                   verbose=2
                                  )

In [ ]:
tic = time()
random_search.fit(X_train, y_train)
toc = time() - tic

In [ ]:
print 'Time:', toc
print random_search.best_score_
print random_search.best_params_

RF:
Time: 4913.59136486
0.348998319125
{'max_leaf_nodes': None, 'bootstrap': False, 'min_samples_leaf': 8, 'n_estimators': 135, 'min_samples_split': 8, 'min_weight_fraction_leaf': 0.1810718592717216, 'max_features': 'sqrt', 'max_depth': 12}

ET:


# Pred

In [ ]:
tic = time()
stack.extra_data['y_binned'] = y_binned
stack.fit(X_train, y_train)
toc = time() - tic

print toc

# Generate submission

In [ ]:
preds = stack.predict(X_test)

In [ ]:
preds_pd = pd.DataFrame({"Id": test_ind, "Hazard": preds})
preds_pd = preds_pd.set_index('Id')
preds_pd.to_csv('submissions/stack_random_crap.csv')